In [26]:
import hashlib
import rdkit as rd
from rdkit import Chem

In [27]:
#Determines if SMILES is valid or not
def is_valid(smiles):
    
    mol = Chem.MolFromSmiles(smiles)

    #Returns True if SMILES is valid, returns False if SMILES is invalid
    return smiles != '' and mol is not None and mol.GetNumAtoms() > 0

In [28]:
completed_lines_hash = set()

#Number of molecules generated
num_smiles_mols = 0

#Number of valid molecules generated
num_valid = 0

#Read in data file line by line
for line in open("generatedmols.txt", "r"):
    
    #Ensure all smiles in original data file are unique
    hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()
  
    if hashValue not in completed_lines_hash:
        completed_lines_hash.add(hashValue)
    
        #Remove \n character
        smiles = line.replace("\n", "")

        #Increment number of valid molecules if SMILES is valid
        if(is_valid(smiles)==True):
            num_valid += 1
    
        #Increment total number of SMILES generated
        num_smiles_mols += 1

In [29]:
print("Total number of SMILES generated:" + str(num_smiles_mols))
print("Number of valid molecules: " + str(num_valid))
print("Model accuracy: {:0.2f}".format(100 * num_valid/num_smiles_mols) + "% of generated SMILES are valid molecules")

Total number of SMILES generated:156
Number of valid molecules: 112
Model accuracy: 71.79% of generated SMILES are valid molecules
